# 1. Import of input data

In [1]:
import nbimporter
import sys
import os
from pathlib import Path

In [2]:
# Import libraries
try:
    current_dir = RUN_DIR
    print(f"📂 Boiler.ipynb executed by RUN.ipynb — using directory: {current_dir}")
except NameError:
    current_dir = Path().resolve()
    print(f"📁 Boiler.ipynb executed standalone — using directory: {current_dir}")

Libraries_path = (current_dir / "Libraries.ipynb").resolve()

# If not found, try in the parent levels
if not Libraries_path.exists():
    Libraries_path = (current_dir.parent / "Libraries.ipynb").resolve()

# Run the file
if Libraries_path.exists():
    print(f"📘 Importing libraries from: {Libraries_path}")
    get_ipython().run_line_magic('run', f'"{Libraries_path}"')
else:
    print("⚠️ Libraries.ipynb not found! Some functions might not be available.")

📁 Boiler.ipynb executed standalone — using directory: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw\Energy_systems
📘 Importing libraries from: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw\Libraries.ipynb
📁 Libraries.ipynb — directory impostata a: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw


In [3]:
# Database parameter import
energysystem = pd.read_excel(energysystem_path, sheet_name = 'Boiler')

# 2. Input parameters

In [4]:
# DASHBORD - VALUE INPUTS
Power = energysystem['Thermal capacity [kW]']
Energy_produced = energysystem['Energy produced [kWh /year]']
Efficiency = energysystem['Efficiency (between 0 and 1)']
EnergyInputSource = energysystem['Energy source (Gas-blend, LPG, Biomethane, Hydrogen)']


In [5]:
# DASHBOARD - INPUT PARAMETERS
Power = Power

Efficiency_LHV = Efficiency

# DEFAULT PARAMETERS
LHV = np.where(EnergyInputSource == 'Gas-blend', 36.8,
      np.where(EnergyInputSource == 'LPG', 93,
      np.where(EnergyInputSource == 'Biomethane', 35,
      np.where(EnergyInputSource == 'Hydrogen', 10.8, np.nan))))

HHV = np.where(EnergyInputSource == 'Gas-blend', 40.4,
      np.where(EnergyInputSource == 'LPG', 102,
      np.where(EnergyInputSource == 'Biomethane', 38,
      np.where(EnergyInputSource == 'Hydrogen', 12.7, np.nan))))

Efficiency_HHV = Efficiency_LHV*(LHV/HHV)  # Efficiency based on HHV

TotEnergyNeeds = np.array([Energy_produced])  # Energy need for 1 year kWh per year

#Define lifespan
Lifespan = np.where(Power <= 70, 19, np.where(Power <= 400, 21, 26))

def calculate_mass(Power):
    if Power < 24.5:
        return 7.430 * np.log(Power) + 11.875
    elif 24.5 <= Power <= 100:
        return 140.86 * np.log(Power) - 415
    else:
        i = np.ceil(Power / 100)
        return (140.86 * np.log(Power / i) - 415) * i

mass = Power.apply(calculate_mass)

# 3. Life cycle inventories

## 3.1 Module A1-3

In [6]:
Boiler_stainlessSteel = 0.156 * mass
Boiler_steel = 0.532 * mass
Boiler_aluminium = 0.128 * mass
Boiler_elastomer = 0.0003 * mass
Boiler_PVC = 0.003 * mass
Boiler_refrigerantgas = 0.065 * mass
Boiler_copper = 0.098 * mass
Boiler_electronic = np.ones(len(Power)) #kg
Boiler_welding = np.where((0 <= Power) & (Power <= 100), 
                          np.full(len(Power), 4),
                          np.where((100 < Power) & (Power <= 1000), 
                                   np.full(len(Power), 6),
                                   np.full(len(Power), 8)
                                  )
                         )

Boiler_waterconsumption = 6.21 * Power + 119.89
Boiler_emission_water_air = 0.015 * Boiler_waterconsumption
Boiler_emission_water_water = 0.002 * Boiler_waterconsumption
Boiler_wastewater_treatment = 0.83 * Boiler_waterconsumption
Boiler_lubricating_oil = 0.27 * Power
Boiler_A13electricity = 0.59 * Power
Boiler_heat = 5.15 * Power
Boiler_hazardous = np.where((0 <= Power) & (Power <= 100), 
                            np.full(len(Power), 0.2),
                            np.where((100 < Power) & (Power <= 1000), 
                                     np.full(len(Power), 0.3),
                                     np.full(len(Power), 0)
                                    )
                           )


## 3.2 Module A4

In [7]:
Boiler_Truk_16_32 = ((500 + 200)*(mass)/1000)
Boiler_Light_Truk = (50*(mass)/1000)

## 3.3 Module B2

In [8]:
Boiler_Light_Truk_B2 = 50 * (Lifespan/2) * 0.01 * (mass/1000)

## 3.4 Module B4

In [9]:
ReqSL = RSP 
BoilerNumberReplacements = (np.ceil(np.divide(ReqSL, Lifespan) - 1))
SFBoiler = (RSP / ReqSL)  # Scaling factor

## 3.5 Module B6

In [10]:
GasBlend   = np.where(EnergyInputSource == 'Gas-blend', 1, 0)
Biomethane = np.where(EnergyInputSource == 'Biomethane', 1, 0)
Hydrogen   = np.where(EnergyInputSource == 'Hydrogen', 1, 0)
LPG        = np.where(EnergyInputSource == 'LPG', 1, 0)

In [11]:
TotEnergyNeeds = TotEnergyNeeds.flatten()

y = (TotEnergyNeeds * 3.6 * RSP).flatten()

# REPARTITION IN THE ENERGY SOURCE
# Costants
k = 0.0256410256410256

# Base terms calculations
base = (1 / Efficiency_HHV) / (k * HHV)
h2_term = (1 / (Efficiency_HHV * (12.7 / 0.09)))

conditions = [
    (EnergyInputSource == 'Gas-blend') & (Year == 2025),
    (EnergyInputSource == 'Gas-blend') & (Year == 2030),
    (EnergyInputSource == 'Gas-blend') & (Year == 2035),
    (EnergyInputSource == 'Gas-blend') & (Year == 2040),
    (EnergyInputSource == 'Gas-blend') & (Year == 2045),
    (EnergyInputSource == 'Gas-blend') & (Year == 2050),
    (EnergyInputSource == 'LPG'),
    (EnergyInputSource == 'Biomethane'),
    (EnergyInputSource == 'Hydrogen')
]

values = [
    base,
    base * 0.90 + base * 0.084 + h2_term * 0.014,
    base * 0.795 + base * 0.189 + h2_term * 0.016,
    base * 0.688 + base * 0.294 + h2_term * 0.018,
    base * 0.582 + base * 0.399 + h2_term * 0.020,
    base * 0 + base * 0.25 + h2_term * 0.20,
    (1 / Efficiency_HHV) / (k * HHV),
    (1 / Efficiency_HHV) / (k * HHV),
    (1 / (Efficiency_HHV * (12.7 / 0.09)))
]

x = np.select(conditions, values, default=np.nan)

Boiler_electricity = 0.0136 * x * y * Grid
Boiler_electricityPV = Boiler_electricity * 0.0056 * PV
Boiler_electricityRECSPV = Boiler_electricity * 0.0056 * RECSPV
Boiler_electricityRECSHydro = Boiler_electricity * 0.0056 * RECSHydro
Boiler_electricityRECSEolic = Boiler_electricity * 0.0056 * RECSEolic

Boiler_EnergyVectorNG = (((1 / (Efficiency_HHV * HHV))) * x * y) * A * GasBlend
Boiler_EnergyVectorBiomethane = ((1 / (Efficiency_HHV * HHV)) * x * y) * B * GasBlend
Boiler_EnergyVectorHydrogen = ((((1 / 0.9) * (1 / 3.6)) * 120) * x * y) * C * GasBlend #120 MJ/kg
Boiler_EnergyVectorBiomethaneInput = (((1 / (Efficiency_HHV * HHV))) * x * y) * Biomethane
Boiler_EnergyVectorLPGinput = (((1 / (Efficiency_HHV * HHV))) * x * y) * LPG
Boiler_EnergyVectorHydrogeninput = ((((1 / 0.9) * (1 / 3.6)) * 120) * x * y) * Hydrogen 

Boiler_Acetaldehyde = 0.000000001* x * y
Boiler_AceticAcid = 0.00000015* x * y
Boiler_Benzene = 0.0000004* x * y
Boiler_Benzoapyrene = 0.00000000001* x * y
Boiler_Butane = 0.0000007* x * y
Boiler_CarbonDioxideFossil = 0.056* x * y
Boiler_CarbonMonoxideFossil = 0.00003* x * y
Boiler_DinitrogenMonoxide = 0.0000005* x * y
Boiler_Formaldehyde = 0.0000001* x * y
Boiler_Mercury = 0.00000000003* x * y
Boiler_MethaneFossil = 0.000002* x * y
Boiler_NitrogenOxides = 0.00002* x * y
Boiler_PAH = 0.00000001* x * y
Boiler_Particulates = 0.0000001* x * y
Boiler_Pentane = 0.0000012* x * y
Boiler_Propane = 0.0000002* x * y
Boiler_PropionicAcid = 0.00000002* x * y
Boiler_SulfurDioxide = 0.0000005* x * y
Boiler_Toluene = 0.0000002* x * y
Boiler_Nitrate = 0.00000013* x * y
Boiler_Nitrite = 0.000000003* x * y
Boiler_Sulfate = 0.00000005* x * y
Boiler_Sulfite = 0.00000005* x * y
Boiler_WatervaporH = 6.79E-01 * x * y
Boiler_NitrogenH = 3.29E+01 * x * y
Boiler_OxygenH = 2.00E+00 * x * y
Boiler_NOXH = 2.83E-03 * x * y
Boiler_WatercondensateH = 8.32E+00 * x * y

## 3.6 Module C2

In [12]:
Boiler_Transport_to_treatment_plant = 50 * (mass/1000)

## 3.7 Module C3

In [13]:
Boiler_EOL_stainlessSteel = Boiler_stainlessSteel
Boiler_EOL_steel = Boiler_steel
Boiler_EOL_aluminium = Boiler_aluminium
Boiler_EOL_elastomer = Boiler_elastomer
Boiler_EOL_PVC = Boiler_PVC
Boiler_EOL_refrigerantgas = Boiler_refrigerantgas
Boiler_EOL_copper = Boiler_copper
Boiler_EOL_electronic = Boiler_electronic #kg
Boiler_EOL_lubricating_oil = Boiler_lubricating_oil #kg
Boiler_EOL_hazardous = Boiler_hazardous #kg

## 3.8 Module C4

In [14]:
Boiler_EOL_landfill_stainlessSteel = Boiler_EOL_stainlessSteel * LandfillMetals2
Boiler_EOL_landfill_steel = Boiler_EOL_steel * LandfillMetals
Boiler_EOL_landfill_aluminium = Boiler_EOL_aluminium * LandfillMetals
Boiler_EOL_inc_elastomer = Boiler_EOL_elastomer * IncinerationElastomer
Boiler_EOL_landfill_elastomer = Boiler_EOL_elastomer * LandfillElastomer
Boiler_EOL_inc_PVC = Boiler_EOL_PVC * IncinerationPVC
Boiler_EOL_landfill_PVC = Boiler_EOL_PVC * LandfillPVC
Boiler_EOL_landfill_refrigerantgas = Boiler_EOL_refrigerantgas * LandfillRefrigerant
Boiler_EOL_landfill_copper = Boiler_EOL_copper * LandfillMetals
Boiler_EOL_inc_electronic = Boiler_EOL_electronic * IncinerationElectronics #kg
Boiler_EOL_landfill_electronic = Boiler_EOL_electronic * LandfillElectronics #kg
Boiler_EOL_inc_lubricatingoil = Boiler_lubricating_oil * IncinerationLubricatingOil #kg
Boiler_EOL_inc_hazardouswaste = Boiler_hazardous * IncinerationHazardous #kg


## 3.9 Module D1 (related to the export of secondary materials)

In [15]:
Boiler_EOL_recycling_stainlessSteel = (RecyclingMetals2 * RecyclingEfficiencyMetals * Boiler_EOL_stainlessSteel) - (RecycledContentMetals * Boiler_EOL_stainlessSteel)
Boiler_EOL_primary_stainlessSteel = ((RecycledContentMetals * Boiler_EOL_stainlessSteel) - (RecyclingMetals2 * RecyclingEfficiencyMetals * Boiler_EOL_stainlessSteel)) * SRmetals

Boiler_EOL_recycling_steel = (RecyclingMetals * RecyclingEfficiencyMetals * Boiler_EOL_steel) - (RecycledContentMetals * Boiler_EOL_steel)
Boiler_EOL_primary_steel = ((RecycledContentMetals * Boiler_EOL_steel) - (RecyclingMetals * RecyclingEfficiencyMetals * Boiler_EOL_steel)) * SRmetals

Boiler_EOL_recycling_aluminium = (RecyclingMetals * RecyclingEfficiencyMetals * Boiler_EOL_aluminium) - (RecycledContentMetals * Boiler_EOL_aluminium)
Boiler_EOL_primary_aluminium = ((RecycledContentMetals * Boiler_EOL_aluminium) - (RecyclingMetals * RecyclingEfficiencyMetals * Boiler_EOL_aluminium)) * SRmetals

Boiler_EOL_recycling_copper = (RecyclingMetals * RecyclingEfficiencyMetals * Boiler_EOL_copper) - (RecycledContentMetals * Boiler_EOL_copper)
Boiler_EOL_primary_copper = ((RecycledContentMetals * Boiler_EOL_copper) - (RecyclingMetals * RecyclingEfficiencyMetals * Boiler_EOL_copper)) * SRmetals


## 3.10 Module D3 (related to the export of energy as a result of waste incineration)

In [16]:
Boiler_EOL_energyrecoveryheat_elastomer = - Boiler_EOL_inc_elastomer * LHVElastomer * Xincheat
Boiler_EOL_energyrecoveryele_elastomer = - Boiler_EOL_inc_elastomer * LHVElastomer * Xincele
Boiler_EOL_energyrecoveryheat_PVC = - Boiler_EOL_inc_PVC * LHVHDPE * Xincheat
Boiler_EOL_energyrecoveryele_PVC = - Boiler_EOL_inc_PVC * LHVHDPE * Xincele
Boiler_EOL_energyrecoveryheat_electronics = - Boiler_EOL_inc_electronic * LHVElectronics * Xincheat
Boiler_EOL_energyrecoveryele_electronics = - Boiler_EOL_inc_electronic * LHVElectronics * Xincele
Boiler_EOL_energyrecoveryheat_lubricatingoil = - Boiler_EOL_inc_lubricatingoil * LHVHazardous * Xincheat
Boiler_EOL_energyrecoveryele_lubricatingoil = - Boiler_EOL_inc_lubricatingoil * LHVHazardous * Xincele
Boiler_EOL_energyrecoveryheat_hazardouswaste = - Boiler_EOL_inc_hazardouswaste * LHVHazardous * Xincheat
Boiler_EOL_energyrecoveryele_hazardouswaste = - Boiler_EOL_inc_hazardouswaste * LHVHazardous * Xincele

# 4. Data for export 

In [17]:
ThermalEnergyProduced_Boiler = Energy_produced * RSP